In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extraction of datset


In [ ]:
import tarfile
import os

# Path to the .tar.gz file in your Drive
tar_gz_path = '/content/drive/MyDrive/AgeGenderProject/cv-corpus-22.0-delta-2025-06-20-en.tar.gz'

# Extract inside your Drive
extract_path = '/content/drive/MyDrive/AgeGenderProject/cv-corpus-22.0'  # Permanent location

os.makedirs(extract_path, exist_ok=True)

with tarfile.open(tar_gz_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

print(f"✅ Extracted to: {extract_path}")



✅ Extracted to: /content/drive/MyDrive/AgeGenderProject/cv-corpus-22.0


Counting total file of mp3

In [ ]:
import os

mp3_dir = '/content/drive/MyDrive/AgeGenderProject/cv-corpus-22.0/en/clips'

mp3_files = [f for f in os.listdir(mp3_dir) if f.endswith('.mp3')]
print(f"🎧 Total .mp3 files: {len(mp3_files)}")


🎧 Total .mp3 files: 26061


Conversion of .mp3 to .wav

In [ ]:
from pydub import AudioSegment
import os
from tqdm import tqdm

mp3_dir = '/content/drive/MyDrive/AgeGenderProject/cv-corpus-22.0/en/clips'
wav_dir = '/content/drive/MyDrive/AgeGenderProject/wav_clips'

# 🔧 Create wav_clips directory if not exists
os.makedirs(wav_dir, exist_ok=True)

# 🔁 Convert all .mp3 files
for file in tqdm(os.listdir(mp3_dir)):
    if file.endswith(".mp3"):
        mp3_path = os.path.join(mp3_dir, file)
        wav_path = os.path.join(wav_dir, file.replace(".mp3", ".wav"))

        try:
            sound = AudioSegment.from_mp3(mp3_path)
            sound.export(wav_path, format="wav")
        except Exception as e:
            print(f"❌ Failed converting {file}: {e}")



100%|██████████| 26061/26061 [2:07:03<00:00,  3.42it/s]


for recheck


In [ ]:
print("👀 Gender values:")
print(df['gender'].value_counts(dropna=False))

print("\n👴 Age values:")
print(df['age'].value_counts(dropna=False))


👀 Gender values:
gender
female_feminine    60
male_masculine     23
Name: count, dtype: int64

👴 Age values:
age
twenties    31
thirties    22
sixties     21
fourties     6
teens        2
eighties     1
Name: count, dtype: int64


In [ ]:
import pandas as pd
import os

# ✅ Load validated.tsv from your Google Drive
tsv_path = '/content/drive/MyDrive/AgeGenderProject/cv-corpus-22.0/en/validated.tsv'
df = pd.read_csv(tsv_path, sep='\t')

# ✅ Filter: valid gender and age
valid_genders = ['female_feminine', 'male_masculine']
df = df[df['gender'].isin(valid_genders) & df['age'].notna()]

# ✅ Normalize gender labels
df['gender'] = df['gender'].map({
    'female_feminine': 'female',
    'male_masculine': 'male'
})

# ✅ Replace .mp3 with .wav
df['path'] = df['path'].str.replace('.mp3', '.wav')

# ✅ Keep only required columns
df = df[['path', 'gender', 'age']]

# ✅ Save cleaned labels to Drive
save_path = '/content/drive/MyDrive/AgeGenderProject/labels.csv'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
df.to_csv(save_path, index=False)

print(f"✅ Extracted & saved: {len(df)} label rows to labels.csv")
df.head()


✅ Extracted & saved: 83 label rows to labels.csv


,path,gender,age
1,common_voice_en_42736613.wav,female,twenties
2,common_voice_en_42798328.wav,female,thirties
7,common_voice_en_42752135.wav,female,twenties
8,common_voice_en_42799557.wav,female,twenties
9,common_voice_en_43193287.wav,male,sixties


MFCC extraction function

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

# ✅ Load cleaned labels
df = pd.read_csv('/content/drive/MyDrive/AgeGenderProject/labels.csv')

# ✅ Path to .wav audio files
wav_folder = '/content/drive/MyDrive/AgeGenderProject/wav_clips'

# ✅ MFCC extraction function
def extract_mfcc(file_path, n_mfcc=13, max_len=200):
    try:
        y, sr = librosa.load(file_path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        # Pad or truncate
        if mfcc.shape[1] < max_len:
            pad_width = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_len]
        return mfcc
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

# ✅ Store features and labels
X, y_gender, y_age = [], [], []

# ✅ Loop through all rows in label file
for i, row in tqdm(df.iterrows(), total=len(df)):
    file_path = os.path.join(wav_folder, row['path'])
    mfcc = extract_mfcc(file_path)
    if mfcc is not None:
        X.append(mfcc)
        y_gender.append(1 if row['gender'] == 'male' else 0)
        y_age.append(row['age'])

# ✅ Convert to NumPy arrays
X = np.array(X)
y_gender = np.array(y_gender)
y_age = np.array(y_age)

# ✅ Save extracted data to your Drive
np.save('/content/drive/MyDrive/AgeGenderProject/features.npy', X)
np.save('/content/drive/MyDrive/AgeGenderProject/gender_labels.npy', y_gender)
np.save('/content/drive/MyDrive/AgeGenderProject/age_labels.npy', y_age)

print(f"\n✅ Features shape: {X.shape}")
print(f"✅ Gender labels shape: {y_gender.shape}")
print(f"✅ Age labels shape: {y_age.shape}")


100%|██████████| 83/83 [00:04<00:00, 19.18it/s]



✅ Features shape: (83, 13, 200)
✅ Gender labels shape: (83,)
✅ Age labels shape: (83,)


In [ ]:
import pandas as pd

# Load labels from your Drive
df = pd.read_csv('/content/drive/MyDrive/AgeGenderProject/labels.csv')

# Display info
print("✅ Total label rows:", len(df))
print("📂 Sample paths:\n", df['path'].head())


✅ Total label rows: 83
📂 Sample paths:
 0    common_voice_en_42736613.wav
1    common_voice_en_42798328.wav
2    common_voice_en_42752135.wav
3    common_voice_en_42799557.wav
4    common_voice_en_43193287.wav
Name: path, dtype: object


Counting label value of age and gende

In [ ]:
import os
import pandas as pd
import numpy as np

# ✅ Load label CSV from Drive
df = pd.read_csv('/content/drive/MyDrive/AgeGenderProject/labels.csv')

# ✅ Check if all .wav files exist
wav_folder = '/content/drive/MyDrive/AgeGenderProject/wav_clips'
missing = [f for f in df['path'] if not os.path.exists(os.path.join(wav_folder, f))]

print(f"❌ Missing .wav files: {len(missing)}")
if missing:
    print("Examples:", missing[:5])

# ✅ Load extracted features and labels from Drive
X = np.load('/content/drive/MyDrive/AgeGenderProject/features.npy')
y_gender = np.load('/content/drive/MyDrive/AgeGenderProject/gender_labels.npy')
y_age = np.load('/content/drive/MyDrive/AgeGenderProject/age_labels.npy', allow_pickle=True)

# ✅ Sanity checks
print("\n✅ Feature shape:", X.shape)  # Expected (n_samples, 13, 200)
print("✅ Gender labels shape:", y_gender.shape)
print("✅ Age labels shape:", y_age.shape)

# 🔎 Show label distributions
print("\n🔢 Gender label distribution:", np.unique(y_gender, return_counts=True))
print("🧓 Age label distribution:", np.unique(y_age, return_counts=True))


❌ Missing .wav files: 0

✅ Feature shape: (83, 13, 200)
✅ Gender labels shape: (83,)
✅ Age labels shape: (83,)

🔢 Gender label distribution: (array([0, 1]), array([60, 23]))
🧓 Age label distribution: (array(['eighties', 'fourties', 'sixties', 'teens', 'thirties', 'twenties'],
      dtype='<U8'), array([ 1,  6, 21,  2, 22, 31]))


All files saved to Google Drive

In [ ]:
import numpy as np

# Save feature and label arrays to your Drive
np.save("/content/drive/MyDrive/AgeGenderProject/X.npy", X)
np.save("/content/drive/MyDrive/AgeGenderProject/y_gender.npy", y_gender)
np.save("/content/drive/MyDrive/AgeGenderProject/y_age.npy", y_age)

print("✅ All files saved to Google Drive!")


✅ All files saved to Google Drive!


Computing Class Weights for Gender Imbalance Handling in Model Training And Model of gender

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import numpy as np

# X shape: (samples, 13, 200) → needs to be (samples, 200, 13) for Conv1D
if X.shape[1] == 13:
    X = np.transpose(X, (0, 2, 1))  # (83, 13, 200) → (83, 200, 13)

# Split the dataset
X_train, X_test, y_gender_train, y_gender_test = train_test_split(
    X, y_gender, test_size=0.2, random_state=42, stratify=y_gender
)

# Compute class weights
gender_classes = np.unique(y_gender_train)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=gender_classes,
    y=y_gender_train
)
gender_class_weight = dict(zip(gender_classes, class_weights))
print("✅ Gender class weights:", gender_class_weight)

# Build the model
gender_model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(200, 13)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
gender_model.fit(
    X_train, y_gender_train,
    validation_data=(X_test, y_gender_test),
    epochs=20,
    batch_size=8,
    class_weight=gender_class_weight,
    verbose=2
)

# Evaluate
loss, acc = gender_model.evaluate(X_test, y_gender_test)
print(f"\n🎯 Gender Accuracy (with class_weight): {acc:.2f}")



✅ Gender class weights: {np.int64(0): np.float64(0.6875), np.int64(1): np.float64(1.8333333333333333)}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
9/9 - 6s - 669ms/step - accuracy: 0.3636 - loss: 0.6896 - val_accuracy: 0.5882 - val_loss: 0.7343
Epoch 2/20
9/9 - 3s - 355ms/step - accuracy: 0.5758 - loss: 0.6570 - val_accuracy: 0.5882 - val_loss: 0.6793
Epoch 3/20
9/9 - 2s - 198ms/step - accuracy: 0.6818 - loss: 0.6730 - val_accuracy: 0.5882 - val_loss: 0.6610
Epoch 4/20
9/9 - 1s - 137ms/step - accuracy: 0.6970 - loss: 0.6749 - val_accuracy: 0.5882 - val_loss: 0.6769
Epoch 5/20
9/9 - 1s - 115ms/step - accuracy: 0.6818 - loss: 0.6839 - val_accuracy: 0.5882 - val_loss: 0.6711
Epoch 6/20
9/9 - 1s - 107ms/step - accuracy: 0.6818 - loss: 0.6577 - val_accuracy: 0.5882 - val_loss: 0.6841
Epoch 7/20
9/9 - 1s - 140ms/step - accuracy: 0.6818 - loss: 0.6173 - val_accuracy: 0.5882 - val_loss: 0.7082
Epoch 8/20
9/9 - 1s - 79ms/step - accuracy: 0.6515 - loss: 0.6550 - val_accuracy: 0.5882 - val_loss: 0.7173
Epoch 9/20
9/9 - 1s - 138ms/step - accuracy: 0.6667 - loss: 0.6353 - val_accuracy: 0.5882 - val_loss: 0.6989
Epoch 10/20
9/9 - 1s

Encode age labels and Model for age

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout

# Load features and age labels
X = np.load("/content/drive/MyDrive/AgeGenderProject/X.npy")
y_age = np.load("/content/drive/MyDrive/AgeGenderProject/y_age.npy", allow_pickle=True)

X = np.load("/content/drive/MyDrive/AgeGenderProject/X.npy")
X = np.transpose(X, (0, 2, 1))  # 🔄 Fix shape mismatch


# Encode age labels
age_encoder = LabelEncoder()
y_age_encoded = age_encoder.fit_transform(y_age)
y_age_cat = to_categorical(y_age_encoded)

# ✅ Filter out rare classes (<2 samples)
unique, counts = np.unique(y_age_encoded, return_counts=True)
valid_classes = [cls for cls, count in zip(unique, counts) if count >= 2]
valid_indices = [i for i, label in enumerate(y_age_encoded) if label in valid_classes]

# Apply filtering
X = X[valid_indices]
y_age_encoded = y_age_encoded[valid_indices]
y_age_cat = y_age_cat[valid_indices]

# Split into train/test sets
X_train, X_test, y_age_train, y_age_test = train_test_split(
    X, y_age_cat, test_size=0.2, random_state=42, stratify=y_age_encoded
)

# 🧠 Build age model
age_model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(200, 13)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(y_age_cat.shape[1], activation='softmax')
])

age_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 🚀 Train
age_model.fit(
    X_train, y_age_train,
    validation_data=(X_test, y_age_test),
    epochs=20,
    batch_size=8
)

# 🎯 Evaluate
loss, acc = age_model.evaluate(X_test, y_age_test)
print(f"\n✅ Age Classification Accuracy: {acc:.2f}")


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 224ms/step - accuracy: 0.2675 - loss: 1.6798 - val_accuracy: 0.4706 - val_loss: 1.4700
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.4210 - loss: 1.5252 - val_accuracy: 0.3529 - val_loss: 1.3067
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.2626 - loss: 1.4846 - val_accuracy: 0.4706 - val_loss: 1.2559
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.4880 - loss: 1.3955 - val_accuracy: 0.4706 - val_loss: 1.2416
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.4323 - loss: 1.4895 - val_accuracy: 0.4706 - val_loss: 1.2263
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.4440 - loss: 1.3497 - val_accuracy: 0.5882 - val_loss: 1.2208
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.4502 - loss: 1.4176 - val_accuracy: 0.4118 - val_loss: 1.2250
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.4320 - loss: 1.4838 - val_accuracy: 0.5882 - val_loss: 1.2306
Epoch 9/20

Augmentation

In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

# ✅ Paths
label_csv = "/content/drive/MyDrive/AgeGenderProject/labels.csv"
orig_path = "/content/drive/MyDrive/AgeGenderProject/wav_clips"
aug_path = "/content/drive/MyDrive/AgeGenderProject/augmented_clips"
os.makedirs(aug_path, exist_ok=True)

# ✅ Load labels
df = pd.read_csv(label_csv)
original_wavs = df['path'].tolist()
y_gender_orig = np.load("/content/drive/MyDrive/AgeGenderProject/gender_labels.npy")
y_age_orig = np.load("/content/drive/MyDrive/AgeGenderProject/age_labels.npy", allow_pickle=True)

# ✅ Build file-to-label mappings
base_to_gender = {}
base_to_age = {}
for i, fname in enumerate(original_wavs):
    base = fname.replace('.wav', '')
    base_to_gender[base] = y_gender_orig[i]
    base_to_age[base] = y_age_orig[i]

# ✅ Augmentation functions
def time_stretch(y, rate=1.1):
    try: return librosa.effects.time_stretch(y, rate)
    except: return y

def pitch_shift(y, sr, n_steps=2):
    try: return librosa.effects.pitch_shift(y, sr, n_steps=n_steps)
    except: return y

def add_noise(y):
    return y + 0.005 * np.random.randn(len(y))

# ✅ Audio settings
n_mfcc = 13
max_len = 200
sr = 16000

# ✅ Store outputs
features = []
gender_labels = []
age_labels = []

# ✅ Process original and augmented clips
for fname in tqdm(original_wavs):
    base = fname.replace('.wav', '')
    full_path = os.path.join(orig_path, fname)

    try:
        y, _ = librosa.load(full_path, sr=sr)
    except:
        print(f"❌ Could not load: {fname}")
        continue

    # Original + augmentations
    versions = [
        (y, f"{base}.wav"),
        (time_stretch(y), f"{base}_stretch.wav"),
        (pitch_shift(y, sr), f"{base}_pitch.wav"),
        (add_noise(y), f"{base}_noise.wav"),
    ]

    for audio, new_name in versions:
        out_path = os.path.join(aug_path, new_name)
        sf.write(out_path, audio, sr)

        # ✅ Extract MFCC
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        if mfcc.shape[1] < max_len:
            pad = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, ((0, 0), (0, pad)), mode='constant')
        else:
            mfcc = mfcc[:, :max_len]

        features.append(mfcc)
        gender_labels.append(base_to_gender[base])
        age_labels.append(base_to_age[base])

# ✅ Convert to arrays
X_aug = np.array(features).transpose((0, 2, 1))  # Shape: (samples, 200, 13)
y_gender_aug = np.array(gender_labels)
y_age_aug = np.array(age_labels)

# ✅ Save to Drive
np.save("/content/drive/MyDrive/AgeGenderProject/X_augmented.npy", X_aug)
np.save("/content/drive/MyDrive/AgeGenderProject/y_gender_augmented.npy", y_gender_aug)
np.save("/content/drive/MyDrive/AgeGenderProject/y_age_augmented.npy", y_age_aug)

print("✅ Augmented dataset saved!")
print("🧠 X_aug shape:", X_aug.shape)
print("🚹 y_gender_aug shape:", y_gender_aug.shape)
print("🎂 y_age_aug shape:", y_age_aug.shape)


100%|██████████| 83/83 [00:18<00:00,  4.50it/s]


✅ Augmented dataset saved!
🧠 X_aug shape: (332, 200, 13)
🚹 y_gender_aug shape: (332,)
🎂 y_age_aug shape: (332,)


Train-test split

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# 📂 Load dataset from Drive
X = np.load("/content/drive/MyDrive/AgeGenderProject/X_augmented.npy")
y_gender = np.load("/content/drive/MyDrive/AgeGenderProject/y_gender_augmented.npy")
y_age = np.load("/content/drive/MyDrive/AgeGenderProject/y_age_augmented.npy", allow_pickle=True)

print("📊 X shape:", X.shape)
print("👫 y_gender shape:", y_gender.shape)
print("🧓 y_age shape:", y_age.shape)

# 🎯 Encode age labels
age_encoder = LabelEncoder()
y_age_encoded = age_encoder.fit_transform(y_age)
y_age_cat = to_categorical(y_age_encoded)

# 🧪 Train-test split (use index-based split to maintain alignment)
indices = np.arange(len(X))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42, stratify=y_gender)

# Use indices to split all arrays
X_train, X_test = X[train_idx], X[test_idx]
y_gender_train, y_gender_test = y_gender[train_idx], y_gender[test_idx]
y_age_train, y_age_test = y_age_cat[train_idx], y_age_cat[test_idx]

print("✅ Data split complete:")
print("📚 X_train:", X_train.shape, " | X_test:", X_test.shape)
print("🚹 y_gender_train:", y_gender_train.shape)
print("🎂 y_age_train (one-hot):", y_age_train.shape)


📊 X shape: (332, 200, 13)
👫 y_gender shape: (332,)
🧓 y_age shape: (332,)
✅ Data split complete:
📚 X_train: (265, 200, 13)  | X_test: (67, 200, 13)
🚹 y_gender_train: (265,)
🎂 y_age_train (one-hot): (265, 6)


For gender

In [ ]:
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout

# ⚖️ Gender class weights
gender_classes = np.unique(y_gender_train)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=gender_classes,
    y=y_gender_train
)
gender_class_weight = dict(zip(gender_classes, class_weights))
print("Gender class weights:", gender_class_weight)

# Build model
gender_model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(200, 13)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
gender_model.fit(
    X_train, y_gender_train,
    validation_data=(X_test, y_gender_test),
    epochs=20,
    batch_size=8,
    class_weight=gender_class_weight
)

# Evaluate
loss, acc = gender_model.evaluate(X_test, y_gender_test)
print(f"🎯 Gender Accuracy: {acc:.2f}")


Gender class weights: {np.int64(0): np.float64(0.6901041666666666), np.int64(1): np.float64(1.8150684931506849)}
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.6396 - loss: 0.7100 - val_accuracy: 0.5075 - val_loss: 0.6973
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.5156 - loss: 0.7559 - val_accuracy: 0.6418 - val_loss: 0.6561
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.7117 - loss: 0.6616 - val_accuracy: 0.5821 - val_loss: 0.6652
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.6971 - loss: 0.6235 - val_accuracy: 0.7612 - val_loss: 0.6113
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 134ms/step - accuracy: 0.5131 - loss: 0.6643 - val_accuracy: 0.7761 - val_loss: 0.5214
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.7050 - loss: 0.6140 - val_accuracy: 0.8358 - val_loss: 0.4399
Epoch 7/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.7107 - loss: 0.5982 - val_accuracy: 0.7910 - val_loss: 0.4464
Epoch 8/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - accuracy: 0.7836 - loss: 0.5035 - val_accuracy: 0.6716 - val_loss: 

For age

In [ ]:
# Build model
age_model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(200, 13)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(y_age_cat.shape[1], activation='softmax')
])

age_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
age_model.fit(
    X_train, y_age_train,
    validation_data=(X_test, y_age_test),
    epochs=20,
    batch_size=8
)

# Evaluate
loss, acc = age_model.evaluate(X_test, y_age_test)
print(f"🎯 Age Accuracy: {acc:.2f}")


Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.2103 - loss: 1.7619 - val_accuracy: 0.3433 - val_loss: 1.4705
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step - accuracy: 0.3811 - loss: 1.4236 - val_accuracy: 0.4627 - val_loss: 1.3397
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.4220 - loss: 1.3718 - val_accuracy: 0.4328 - val_loss: 1.3592
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - accuracy: 0.3894 - loss: 1.3769 - val_accuracy: 0.3881 - val_loss: 1.3361
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.4536 - loss: 1.2987 - val_accuracy: 0.3731 - val_loss: 1.3122
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.3887 - loss: 1.4247 - val_accuracy: 0.4776 - val_loss: 1.2646
Epoch 7/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.4708 - loss: 1.2620 - val_accuracy: 0.3881 - val_loss: 1.2870
Epoch 8/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.4152 - loss: 1.3117 - val_accuracy: 0.4776 -

Attention

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, Dense,
    GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention, Add
)

# ✅ Input shape: (200 time steps, 13 MFCC features)
input_layer = Input(shape=(200, 13))

# 🔹 Convolutional feature extraction
x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.3)(x)

# 🔹 Another conv block
x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.3)(x)

# 🔸 Self-Attention block
# Normalize
norm = LayerNormalization()(x)

# Multi-head attention (2 heads is enough for MFCCs)
attn_out = MultiHeadAttention(num_heads=2, key_dim=64)(norm, norm)
x = Add()([x, attn_out])  # Residual connection

# 🔹 Global Average Pooling
x = GlobalAveragePooling1D()(x)

# 🔹 Dense classifier
x = Dense(64, activation='relu')(x)
output_layer = Dense(y_age_cat.shape[1], activation='softmax')(x)

# 🧠 Build model
age_attention_model = Model(inputs=input_layer, outputs=output_layer)
age_attention_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

age_attention_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 200, 13)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 200, 64)   │      2,560 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_7     │ (None, 100, 64)   │          0 │ conv1d_7[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 100, 64)   │          0 │ max_pooling1d_7[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 100, 128)  │     24,704 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_8     │ (None, 50, 128)   │          0 │ conv1d_8[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 50, 128)   │          0 │ max_pooling1d_8[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 50, 128)   │        256 │ dropout_15[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 128)   │     66,048 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 50, 128)   │          0 │ dropout_15[0][0], │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ add[0][0]         │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 64)        │      8,256 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 6)         │        390 │ dense_14[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 102,214 (399.27 KB)

 Trainable params: 102,214 (399.27 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = age_attention_model.fit(
    X_train, y_age_train,
    validation_data=(X_test, y_age_test),
    epochs=30,
    batch_size=8,
    verbose=1
)

# Evaluate
loss, acc = age_attention_model.evaluate(X_test, y_age_test)
print(f"🎯 Age Accuracy (CNN+Attention): {acc:.2f}")

Epoch 1/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9356 - loss: 0.1800 - val_accuracy: 0.8358 - val_loss: 0.6660
Epoch 2/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9802 - loss: 0.0762 - val_accuracy: 0.8507 - val_loss: 0.3801
Epoch 3/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9695 - loss: 0.0708 - val_accuracy: 0.8955 - val_loss: 0.2814
Epoch 4/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9927 - loss: 0.0348 - val_accuracy: 0.9552 - val_loss: 0.1973
Epoch 5/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9788 - loss: 0.0458 - val_accuracy: 0.9552 - val_loss: 0.1551
Epoch 6/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9772 - loss: 0.0526 - val_accuracy: 0.9552 - val_loss: 0.1612
Epoch 7/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9978 - loss: 0.0264 - val_accuracy: 0.9552 - val_loss: 0.1381
Epoch 8/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.9789 - loss: 0.0553 - val_accuracy: 0.9403 - v